In [1]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
import pandas as pd

Using TensorFlow backend.


In [2]:
sl_train = pd.read_csv('sign_mnist_train.csv')
sl_test = pd.read_csv('sign_mnist_test.csv')
sl=pd.concat([sl_train, sl_test])
sl.reset_index(inplace=True, drop=True)
sl.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [3]:
x=sl.drop(columns=['label'])
y=sl['label']

In [4]:
(len(x.columns),len(x))

(784, 34627)

In [5]:
tr = LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
y = tr.fit_transform(y)

In [6]:
y[0:10]

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [7]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [8]:
x[0:10]

array([[-0.92873509, -0.76578328, -0.62705214, ...,  0.67180891,
         0.66447665,  0.66237412],
       [ 0.21852785,  0.20063842,  0.10982451, ..., -0.90899662,
        -0.39151294, -0.14986952],
       [ 0.98336982,  0.96881976,  0.92292978, ...,  0.53094506,
         0.52471333,  0.55509665],
       ...,
       [-0.76142591, -2.64906658, -1.97375775, ...,  1.03179433,
         1.05270812,  1.06083326],
       [ 0.55314621,  0.62189915,  0.61801531, ..., -1.36289128,
        -1.54067809, -1.55980187],
       [ 1.03117244,  0.99359981,  0.94833932, ...,  0.21791426,
         0.27624518, -0.04259206]])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [10]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10, min_delta=1)
result = pd.DataFrame(columns=['accuracy','loss','validation_split','units','metrics'])
model=[]
for vals in [0.2,0.25,0.3]:
    for unit in [25,50,100,500]:
        for acc in ['accuracy','categorical_accuracy']:
            cl = Sequential()
            cl.add(Dense(units=unit, activation='relu', use_bias=True, kernel_initializer='uniform', bias_initializer='zeros', input_shape=(x_train.shape[1],)))
            cl.add(Dense(units=unit, activation='relu', use_bias=True, kernel_initializer='uniform', bias_initializer='zeros', input_shape=(x_train.shape[1],)))
            cl.add(Dense(units=24, activation='softmax', use_bias=True, kernel_initializer='uniform', bias_initializer='zeros'))
            cl.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[acc])
            cl.fit(x_train, y_train, epochs=1000, batch_size=10, validation_split=vals, verbose=0, callbacks=[es])
            eva = cl.evaluate(x_test, y_test, batch_size=128, verbose=0)
            model.append(cl)
            result = result.append({'accuracy':eva[1],'loss':eva[0],'validation_split':vals,'units':unit,'metrics':acc},ignore_index=True)

In [11]:
result.sort_values(by=['accuracy','loss'], ascending=[False,True], inplace=True)
result

,accuracy,loss,validation_split,units,metrics
13,1.000000,0.000003,0.25,100,categorical_accuracy
5,1.000000,0.000013,0.20,100,categorical_accuracy
21,1.000000,0.000031,0.30,100,categorical_accuracy
4,1.000000,0.000044,0.20,100,accuracy
18,1.000000,0.000086,0.30,50,accuracy
2,1.000000,0.000103,0.20,50,accuracy
20,1.000000,0.000104,0.30,100,accuracy
3,1.000000,0.000207,0.20,50,categorical_accuracy
11,1.000000,0.000247,0.25,50,categorical_accuracy
16,1.000000,0.001139,0.30,25,accuracy
